# IRS RAG Pipeline with Sentence Chunking

In [ ]:
# # Fix for Google Colab to avoid punkt_tab errors
# !pip install --upgrade --force-reinstall nltk

# import nltk
# nltk.download("punkt", download_dir="/content/nltk_data")
# nltk.data.path.append("/content/nltk_data")

# # Sanity check
# from nltk.tokenize import sent_tokenize
# print(sent_tokenize("This is a test. Here's another sentence."))


In [1]:
# If running in Colab, uncomment:
!pip install PyMuPDF tqdm sentence-transformers faiss-cpu pandas transformers gradio nltk
!pip install -U accelerate bitsandbytes transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211

In [2]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import re
import fitz
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from nltk.tokenize import sent_tokenize

pdf_folder = "/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/Rag model data/irs_eng_pdfs"

In [6]:
def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    return text.replace("\n", " ").strip()

def split_list(input_list: list, slice_size: int) -> list:
    """Splits a list into chunks of a given size."""
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

num_sentence_chunk_size = 10
pages_and_chunks = []

for filename in tqdm(os.listdir(pdf_folder)):
    if filename.endswith(".pdf"):
        file_path = os.path.join(pdf_folder, filename)
        file_type = (
            "form" if filename.lower().startswith("f") else
            "instruction" if filename.lower().startswith("i") else
            "publication" if filename.lower().startswith("p") else
            "unknown"
        )
        try:
            doc = fitz.open(file_path)
            for page_number, page in enumerate(doc):
                raw_text = page.get_text()
                formatted_text = text_formatter(raw_text)
                sentences = sent_tokenize(formatted_text)
                sentence_chunks = split_list(sentences, num_sentence_chunk_size)
                for chunk in sentence_chunks:
                    joined_chunk = " ".join(chunk).replace("  ", " ").strip()
                    joined_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_chunk)
                    pages_and_chunks.append({
                        "file": filename,
                        "file_type": file_type,
                        "page_number": page_number + 1,
                        "sentence_chunk": joined_chunk,
                        "chunk_char_count": len(joined_chunk),
                        "chunk_word_count": len(joined_chunk.split(" ")),
                        "chunk_token_count": len(joined_chunk) / 4
                    })
        except Exception as e:
            print(f"Error reading {filename}: {e}")

df = pd.DataFrame(pages_and_chunks)
df.to_csv("/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/sentence_chunks_df.csv", index=False)
df.head()


  0%|          | 0/2214 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
# Loading saved csv
csv_save_path = "/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/sentence_chunks_df.csv"
df = pd.read_csv(csv_save_path)
df.head()

,file,file_type,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,p5633.pdf,publication,1,OVER-THE-PHONE INTERPRETER (OPI) Languages Rep...,1039,126,259.75,[-7.49532208e-02 3.27443853e-02 -3.30116227e-...
1,f14824.pdf,form,1,Form 14824 (Rev. 10-2022) Catalog Number 69954...,2258,401,564.50,[-9.38651860e-02 4.72247507e-03 6.19203560e-...
2,f14824.pdf,form,1,"• In either case, to show where you lived, you...",1956,332,489.00,[-9.33274254e-02 1.76005848e-02 9.84594505e-...
3,f14824.pdf,form,2,Form 14824 (Rev. 10-2022) Catalog Number 69954...,501,81,125.25,[-3.82399783e-02 1.47773055e-02 2.13823803e-...
4,i109495c.pdf,instruction,1,2024 Instructions for Forms 1094-C and 1095-C ...,1094,149,273.50,[-7.46698380e-02 1.06692538e-01 -1.70740671e-...


# **Embedding our text chunks**

In [6]:
from sentence_transformers import SentenceTransformer
import torch

In [17]:
# Check for GPU and set appropriate precision
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32  # Use FP16 on GPU

# Load the optimized Sentence Transformer model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2",
                            device=device,
                            trust_remote_code=True)

# Increase batch size for efficient GPU utilization
BATCH_SIZE = 128 if device == "cuda" else 32

# Generate embeddings in batches
text_chunks = df["sentence_chunk"].tolist()
embeddings = model.encode(text_chunks,
                          batch_size=BATCH_SIZE,
                          convert_to_tensor=True,
                          device=device)  # Keeps data on GPU for faster computation

# Convert embeddings to NumPy array (if needed)
df["embedding"] = [embedding.cpu().numpy() for embedding in embeddings]


In [ ]:
import faiss

# Convert embeddings to NumPy array
embedding_matrix = np.array(df["embedding"].tolist()).astype('float32')

# Create FAISS index (for L2/Euclidean search)
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)

# Save FAISS index
faiss.write_index(index, "/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/faiss_index.bin")

df.to_csv("/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/sentence_chunks_df.csv", index=False)

print(" Processing Completed! FAISS Index & CSV Saved.")


Processing Completed! FAISS Index & CSV Saved.


In [18]:
import faiss

# Load the FAISS index
index = faiss.read_index("/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/faiss_index.bin")

In [19]:
# **Function for Similarity Search**
def search_similar_text(query_text, top_k=3):
    query_embedding = model.encode([query_text], convert_to_tensor=True).cpu().numpy()
    D, I = index.search(query_embedding, k=top_k)
    similar_chunks = df.iloc[I[0]]["sentence_chunk"].tolist()
    return similar_chunks

# Example
query = "How do I file my tax returns?"
print("Similar Chunks Found:\n", search_similar_text(query))

 Similar Chunks Found:
 ['Preparing and filing your tax return. After receiving all your wage and earnings state- ments (Forms W-2, W-2G, 1099-R, 1099-MISC, 1099-NEC, etc. ); unemployment compensation statements (by mail or in a digital format) or other government payment statements (Form 1099-G); and interest, dividend, and retirement statements from banks and investment firms (Forms 1099), you have several options to choose from to prepare and file your tax return. You can prepare the tax return yourself, see if you qualify for free tax preparation, or hire a tax professional to prepare your return. Free options for tax preparation. Go to IRS.gov to see your options for preparing and filing your return online or in your local commun- ity, if you qualify, which include the following. • Direct File. Direct File is a permanent op- tion to file individual federal tax returns on- line—for free—directly and securely with the iRS. Direct File is an option for taxpay- ers in participating st

In [20]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
use_quantization_config = False # Using low GPU memory
if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
  attn_implementation = "flash_attention_2"
else:
  attn_implementation = "sdpa"
  print(attn_implementation)
print(f"[INFO] Using attention implementation: {attn_implementation}")

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

llm_model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype=torch.float16,
                                             quantization_config=quantization_config if use_quantization_config else None,
                                             low_cpu_mem_usage=False,
                                             attn_implementation=attn_implementation)
if not use_quantization_config:
    llm_model.to("cuda")



sdpa
[INFO] Using attention implementation: sdpa


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [21]:
def augment_prompt(query_text, context_list):
  context = "- " + "\n- ".join([item for item in context_list])

  # base_prompt = """Based on the following context items, please answer the query.
  # Give yourself room to think by extracting relevant passages from the context before answering the query.
  # Don't return the thinking, only return the answer.
  # Make sure your answers are as explanatory as possible.
  # Use the following examples as reference for the ideal answer style.
  # \nExample 1:
  # Query:
  # Answer:
  # \nExample 2:
  # Query:
  # Answer:
  # \nExample 3:
  # Query:
  # Answer:
  # \nNow use the following context items to answer the user query:
  # {context}
  # \nRelevant passages: <extract relevant passages from the context here>
  # User query: {query}
  # Answer:"""

  base_prompt = """Based on the following context items, please answer the query.
  Give yourself room to think by extracting relevant passages from the context before answering the query.
  Don't return the thinking, only return the answer.
  Make sure your answers are as explanatory as possible.
  \nNow use the following context items to answer the user query:
  {context}
  \nRelevant passages: <extract relevant passages from the context here>
  User query: {query}
  Answer:"""


  base_prompt = base_prompt.format(context=context, query=query_text)
  dialogue_template = [
        {"role": "user",
        "content": base_prompt}
    ]
  prompt = tokenizer.apply_chat_template(dialogue_template, tokenize=False, add_generation_prompt=True)
  return prompt

In [28]:
input_text = "Who should file form 943?"

similar_chunks = search_similar_text(input_text, 5)
augmented_prompt = augment_prompt(input_text, similar_chunks)

In [29]:
print(augmented_prompt)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Based on the following context items, please answer the query.
  Give yourself room to think by extracting relevant passages from the context before answering the query.
  Don't return the thinking, only return the answer.
  Make sure your answers are as explanatory as possible.
  
Now use the following context items to answer the user query:
  - . . . . 4 Who Must File Form 943? . . . . .
- . . . . 3 Who Must File Form 8962 . . . . . .
- . . 6 Where Should You File Form 943-X? . . . . . . .
- . . . 5 When Should You File Form 943-X? . . . . . .
- . . . 2 Who Needs To File Form W-2 and Form W-3? . . . . . .
  
Relevant passages: <extract relevant passages from the context here>
  User query: Who should file form 943?
  Answer:<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [30]:
inputs = tokenizer(augmented_prompt, return_tensors="pt").to('cuda')
output_prompt = llm_model.generate(**inputs, temperature=0.7, do_sample=True, max_new_tokens=512)
outputs_decoded = tokenizer.decode(output_prompt[0])
print(f"Model output (decoded):\n{outputs_decoded}\n")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Model output (decoded):
<|begin_of_text|><|begin_of_text|><|start_header_id|>user<|end_header_id|>

Based on the following context items, please answer the query.
  Give yourself room to think by extracting relevant passages from the context before answering the query.
  Don't return the thinking, only return the answer.
  Make sure your answers are as explanatory as possible.
  
Now use the following context items to answer the user query:
  -.... 4 Who Must File Form 943?.....
-.... 3 Who Must File Form 8962......
-.. 6 Where Should You File Form 943-X?.......
-... 5 When Should You File Form 943-X?......
-... 2 Who Needs To File Form W-2 and Form W-3?......
  
Relevant passages: <extract relevant passages from the context here>
  User query: Who should file form 943?
  Answer:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Relevant passages:

* ".... 4 Who Must File Form 943?......"
* ".... 2 Who Needs To File Form W-2 and Form W-3?......"

Answer: An employer who pays wage